# Tutorial: Sequence diversity metrics using rTASSEL

## Enter your notebook title here

**Objective**: Enter your objective here  
**Data**: Describe your data set here  
**User and contact**: Enter your name and contact here

### Table of contents
* [Notes](#Notes)
* [Libraries](#Libraries)
* [Parameters and functions](#Parameters-and-functions)
* [Data](#Data)
    * [Retrieve BrAPI data and filter](#Retrieve-BrAPI-data-and-filter)
        * [Inspect genotype data in R](#Inspect-genotype-data-in-R)
        * [Load genotype data into rTASSEL](#Load-genotype-data-into-rTASSEL)
* [Analysis](#Analysis)
    + [Sequence diversity](#Sequence-diversity)
* [Visualization](#Visualization)
* [References and additional resources](#References-and-additional-resources)

## Notes

This tutorial assumes: 
1. You already know how to load your data via a BrAPI database into rTASSEL and will inspect your data:
    - See 01_rTASSEL_load_data.brapi.ipynb on how to load files via BrAPI databases.
2. You will filter your genotype data as appropriate for your data set and analysis:
    - See 02_rTASSEL_GenotypeFiltering.brapi.ipynb for a tutorial on how to filter genotype data when retrieving data via BrAPI and using rTASSEL.

Additional documentation on the `seqDiversity()` function in rTASSEL can be found [here](https://rtassel.maizegenetics.net/reference/seqDiversity.html).

In [ ]:
getwd()

In [ ]:
Sys.Date()

## Libraries

In [ ]:
library(QBMS)
library(rTASSEL)
library(ggplot2)
library(dplyr)

## Parameters and functions

Create a function for setting the dimensions of a plot:

In [ ]:
fig <- function(width, heigth) {
    options(
        repr.plot.width  = width, 
        repr.plot.height = heigth
    )
}

## Data

### Retrieve BrAPI data and filter

**You will need to log into Gigwa using the BrAPI helper.**

In [ ]:
geno_provider$gigwa_list_dbs()

**Please edit the code to set your database (db):**

In [ ]:
geno_provider$gigwa_set_db("myDataBase")

In [ ]:
geno_provider$gigwa_list_projects()

**Please edit the code to set your project:**

In [ ]:
geno_provider$gigwa_set_project("myProject")

**Edit the below code to use appropriate filters for your data set and analysis, additional filtering can be done after retrieving the data and loading into rTASSEL.** 

In [ ]:
genoDataFromGigwa <- geno_provider$gigwa_get_variants(
    max_missing = 0.2,
    min_maf = 0.05
)

#### Inspect genotype data in R

In [ ]:
genoDataFromGigwa |> head()
genoDataFromGigwa |> dim()
genoDataFromGigwa |> names()

#### Load genotype data into rTASSEL 

In [ ]:
tasGeno <- genoDataFromGigwa |> rTASSEL::readGenotypeTableFromGigwa()

In [ ]:
tasGeno

**Perform additional filtering steps in rTASSEL for your data set and analysis:**  
    - See 02_rTASSEL_GenotypeFiltering.brapi.ipynb for more details about filtering.

## Analysis

### Sequence diversity 

In rTASSEL `seqDiversity()` provides: 

- segregating sites
- average pairwise divergence (𝜋) 
- estimated mutation rate (𝜃 or 4𝑁𝜇)  
- Tajima's D 

By default `seqDiversity` will calculate diversity metrics across the entire set of sites in the genotype data, providing a single set of diversity metrics for all markers:  

Diversity metrics are returned in `$Diversity`.  
Polymorphic distribution is returned in `$PolyDist`.

In [ ]:
tasGeno |> seqDiversity()

Options can be provided to change how the diversity metrics are calculated:

- using `startSite` and `endSite` to restrict the analysis or,

- by creating a `sliding window analysis` as in the below example:

In [ ]:
seqRestrict <- tasGeno |>
    seqDiversity(
        slidingWindowAnalysis = TRUE,
        stepSize = 50,
        windowSize = 100
    )

In [ ]:
seqRestrict$Diversity |> head()

## Visualization

Visualize sequence diversity across the genome using `ggplot()`

First, set the plot dimensions with the figure function created at the start of the notebook:

In [ ]:
fig(12,5)

In this example, Tajima's D is plotted for chromosome 1 as calculated in the sliding sindow analysis created above:

In [ ]:
seqRestrict$Diversity |>
    filter(Chromosome == "1") |>
    ggplot() +
    aes(x = StartChrPosition, y = TajimaD) +
    geom_line()

## References and additional resources

To cite rTASSEL, please use the following citation:

Monier et al., (2022). rTASSEL: An R interface to TASSEL for analyzing genomic diversity. Journal of Open Source Software, 7(76), 4530, https://doi.org/10.21105/joss.04530.

You can find more information about rTASSEL [here](https://rtassel.maizegenetics.net)

and an rTASSEL tutorial in binder [here](https://mybinder.org/v2/gh/btmonier/rTASSEL_sandbox/HEAD?labpath=getting_started.ipynb).

**Please also cite QBMS using the following citation:**

Al-Shamaa K (2023). QBMS: Query the Breeding Management System(s). R package version 0.9.1, https://icarda-git.github.io/QBMS/.